In [1]:
from keras.optimizers import Adam
import keras
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import img_to_array
from keras.utils import to_categorical
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D, Convolution2D
import numpy as np
import random

import cv2
import os
import pickle
if not os.path.exists('pickle_data'):
    os.makedirs('pickle_data')
print("Start")

Using TensorFlow backend.


Start


In [2]:
print("loading images...")
data = []
labels = []
if not os.path.exists('pickle_data/melanoma_data.pkl'):
    directory = os.fsencode("melanoma")
    for file in os.listdir(directory):
        filename = os.fsdecode(file)
        path = str('melanoma/' + filename)
        image = cv2.imread(path)
        image = cv2.resize(image, (64, 64))
        image = img_to_array(image)
        data.append(image)
        labels.append(1)

    directory = os.fsencode("others")
    for file in os.listdir(directory):
        filename = os.fsdecode(file)
        path = str('others/' + filename)
        image = cv2.imread(path)
        image = cv2.resize(image, (64, 64))
        image = img_to_array(image)
        data.append(image)
        labels.append(0)
    d = {}
    d['data'] = data
    d['labels'] = labels
    pickle.dump(d, open('pickle_data/melanoma_data.pkl', 'wb'))
else:
    d = pickle.load(open( 'pickle_data/melanoma_data.pkl', 'rb'))
    data = d['data']
    labels = d['labels']
                
print("Done.")

loading images...
Done.


In [3]:
data = np.array(data, dtype="float") / 255.0
labels = np.array(labels)

(x_train, x_test, y_train, y_test) = train_test_split(data,labels, test_size=0.20, random_state=42)
y_train = to_categorical(y_train, num_classes=2)
y_test = to_categorical(y_test, num_classes=2)
print("e")

e


In [4]:
batch_size = 32
num_classes = 2
epochs = 10

img_rows, img_cols = 64, 64

input_shape = (img_rows, img_cols,3)

print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),activation='relu',input_shape=input_shape))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy, optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

#model.compile(loss='binary_crossentropy',optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),metrics=['accuracy'])


model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

x_train shape: (1600, 64, 64, 3)
1600 train samples
400 test samples
Train on 1600 samples, validate on 400 samples
Epoch 1/10
1600/1600 [==============================] - 11s 7ms/step - loss: 0.5198 - acc: 0.8094 - val_loss: 0.4779 - val_acc: 0.8050